In [2]:
! pip install pysam

    100% |████████████████████████████████| 2.3MB 484kB/s ta 0:00:01
  Running setup.py bdist_wheel for pysam ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \

In [ ]:
import pysam
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from genepeeks.common import utilities as util
import seaborn as sns
import mpmath 
mpmath.mp.dps = 100  # set precision

%matplotlib inline

/Users/nigel/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
DMD_ensembl = util.Mongo.get_collection_data('gene', wanted_db='prod', query={'_id': 'DMD'}, find_one=True, single_field='ensembl')
DMD_exons = util.get_nested_value(DMD_ensembl, ('is_primary', 'transcripts', 'is_primary', 'exons'))
# had to make local change in utilities 
DMD_exons_merged = util.merge_intervals(DMD_exons, min_dist=200, include_index=True)
exon_labels = ['Ex' + exon['index'] for exon in DMD_exons_merged]

In [3]:
# DMD exon/gene coordinates and useful regions (coordinates taken from IGV b37 reference)
EXON46 = [31950197, 31950344]
EXON47 = [31947713, 31947862]
EXON48 = [31893308, 31893490]
EXON49 = [31854835, 31854939]
EXON50 = [31838092, 31838200]
EXON51 = [31792077, 31792309]
DMD = [31115794, 33357558]  # taken from ensembl GRCh37

# deletion of partial exons
EX48_50_PART = [31838130, 31893308]

# for consistency with exon labeling in Ensembl
exon_dict = {exon['index']: [exon['start'], exon['end']] for exon in DMD_exons_merged}
print exon_dict['1'][1] - exon_dict['1'][0]

2702


In [ ]:
# workaround to force real time print output in notebooks
# see http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

## Starting Data

In [ ]:
coverage_df = pd.read_csv('../exon_data/coverage_matrix.csv', header=1, index_col=0)
coverage_df.index.name = None
coverage_df.date_modified = pd.to_datetime(coverage_df.date_modified)
coverage_df_f = coverage_df[coverage_df.gender == 'F']
coverage_df_m = coverage_df[coverage_df.gender == 'M']
coverage_df_RMA = coverage_df[coverage_df.subject.str.contains('FRMR')]
coverage_df_M1 = coverage_df[coverage_df.sequencer == 'M1']
coverage_df_M1_f = coverage_df_f[coverage_df_f.sequencer == 'M1']
coverage_df_T1_f = coverage_df_f[coverage_df_f.sequencer == 'T1']
print coverage_df.shape
coverage_df.head()

In [ ]:
# function for reshaping data frames so that exons are observations (rows) and subjects are variables (columns)
def reshape_df(df, include_stats=False, subject_droplist=None):
    df_nodate = df.drop('date_modified', axis=1)
    df_grouped = df_nodate.groupby(['subject']).sum()
    df_norm = df_grouped.div(df_grouped.sum(axis=1), axis=0)
    df_norm = df_norm.transpose().reset_index()
    df_norm.rename(columns={'index': 'Exon'}, inplace=True)

    if subject_droplist:
        for subject in subject_droplist:
            df_norm.drop(subject, axis=1, inplace=True)
    if include_stats:
        df_norm['Mean'] = df_norm.mean(axis=1)
        df_norm['SD'] = df_norm.std(axis=1)
    return df_norm

In [18]:
# use RMA samples for initial probability vector
# this is only the females in RMA
RMA_norm = reshape_df(coverage_df_RMA, include_stats=True)
RMA_norm.head()
X_probs = np.array(RMA_norm.Mean)
print len(X_probs)
X_probs

78


array([ 0.04589008,  0.00181261,  0.0111598 ,  0.01918853,  0.01104113,
        0.0166613 ,  0.0166843 ,  0.01212952,  0.00371631,  0.01597643,
        0.01188   ,  0.00994347,  0.00875295,  0.00676731,  0.0183255 ,
        0.007394  ,  0.00961452,  0.00514529,  0.01048072,  0.02985736,
        0.01276754,  0.01643603,  0.01196839,  0.00946219,  0.02820376,
        0.01037773,  0.02059615,  0.0089391 ,  0.01577857,  0.00666614,
        0.009583  ,  0.00995351,  0.02010135,  0.00837557,  0.01176132,
        0.0112656 ,  0.01252437,  0.00844386,  0.01113459,  0.01235549,
        0.01049119,  0.0076502 ,  0.01632141,  0.01293299,  0.02815898,
        0.00908641,  0.0234396 ,  0.01244374,  0.0216019 ,  0.00885256,
        0.01000741,  0.00725296,  0.02025866,  0.00825177,  0.02257343,
        0.00719029,  0.00731126,  0.00843966,  0.01311383,  0.02040325,
        0.00818417,  0.01599219,  0.00793946,  0.01305769,  0.02064807,
        0.01183257,  0.01847446,  0.00963051,  0.01236458,  0.00

## Gibbs Sampling


The full gene product of DMD consists of 78 exons, each with its own copy number in each subject ($\mathbf{C} = \{c_1, c_2, ..., c_{78}\}$), where $c_i \in{1,2,3...}$ For this simple model, assume that the $c_i$ are distributed discrete uniformly with support $ z = \{1,2,3\}$. 

We also know "intensity" values across the exons $\mathbf{X}= \{x_1, x_2,..., x_{78}\}$, and we let $$\mathbf{P} = \dfrac{\mathbf{C}\mathbf{X}}{\sum_i c_ix_i} = \{p_1, p_2, ..., p_{78}\}$$ 

This generates probability values for the multinomial distribution from which we will sample the reads for a single simulated "sequencing run".
 


Initialize the intensities $\mathbf{X}$ (see X_probs above). Initialize $c_i$ values based on basic prior distribution. Assume 5000 reads per sample/subject and initialize a single sequencing run $Y^0 = \{y_1, y_2, ... y_{78}\}$. 

At each iteration $t$, sample $c_i^t$ from the posterior distribution
$$f_{C^t|Y, C_j^t}(c) = \dfrac{\Pi_j (p_j|c_i=c)^{y_j}}{\sum_c \Pi_j (p_j|c_i=c)^{y_j}} $$ 
where $$ C_j^t = \{ c_1^t,...c_{i-1}^t, c_{i+1}^{t-1}, ... c_{78}^{t-1} \}$$

Normally this expression would include the prior distribution over $c$ but it easily cancels out as the uniform. (This expression also does not include the multinomial constant). 

After setting $c_i^t$, sample new sequencing run $Y$. Alternatively, could sample new sequencing run $Y^t$ only once per iteration $t$. 

Implementation considerations: $\Pi_j (p_j|c_i=c)^{y_j}$ generates underflow in Python, but trying to use the log of the posterior distribution leads to 
$$  \sum y_j \log (p_j|c_i=c) - \log \sum \Pi_j (p_j|c_i=c)^{y_j}$$

Potential solution: Setting $c_i^t$ to $c$ value with highest log-likelihood is decent approximation when there is significant difference in likelihoods, but not very accurate otherwise.

In [6]:
cnv = np.random.randint(low=1, high=4, size=78)
normed_probs = np.multiply(cnv, X_probs) / sum(np.multiply(cnv, X_probs)) 

NameError: name 'np' is not defined

In [25]:
normed_probs = np.multiply(cnv, X_probs) / sum(np.multiply(cnv, X_probs)) 

In [77]:
cnv = 2 * np.ones(78)
normed_probs = np.multiply(cnv, X_probs) / sum(np.multiply(cnv, X_probs)) 
data = np.random.multinomial(5000, normed_probs)
data

array([228,  10,  72, 102,  51,  69,  98,  69,  18,  68,  51,  55,  46,
        32,  93,  37,  52,  15,  55, 145,  62,  87,  70,  56, 127,  43,
        92,  45,  82,  34,  53,  51, 107,  41,  54,  53,  64,  34,  54,
        66,  52,  40,  73,  53, 147,  50, 115,  67, 108,  35,  42,  34,
       114,  54, 122,  39,  35,  35,  73,  91,  53,  87,  46,  68, 111,
        49,  91,  47,  56,  28,  61,  37,  74,  19,  28,  66,  13,  46])

In [169]:
cnv = 2 * np.ones(78)
normed_probs = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs)) 
data = np.random.multinomial(5000, normed_probs)

values = []
pdf = []
for i in [1,2,3]:
    cnv[5] = i
    normed_probs = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs)) 
    log_likelihood =  np.sum(np.multiply(np.log(normed_probs), data))
    print log_likelihood
    values.append(np.prod(1e120 * np.power(normed_probs, data)))
    mpf = mpmath.fprod(np.power(normed_probs, data))
    print mpf
    pdf.append(mpf)
    print np.prod(1e120 * np.power(normed_probs, data))
probs = values / np.sum(values)
print mpmath.fsum(pdf)
pdf = mpmath.matrix(pdf) / mpmath.fsum(pdf)
pdf_list = np.array(pdf.tolist(), dtype=float)
pdf_list = [x[0] for x in pdf_list]
print pdf_list
print np.random.choice(cnv_support, 1, p=pdf_list)[0]

-21149.7365731
0.0
0.0
-21137.4988167
0.0
0.0
-21147.3532511
0.0
0.0
0.0


/Users/velina/.pyenv/versions/2.7.11/envs/dmd/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in divide


ZeroDivisionError: 

In [150]:
random.seed(24)
total_reads = 5000
cnv_support = [1,2,3]
cnv = np.random.randint(low=1, high=4, size=78)
use_log = True
print cnv
cnv2 = 2 * np.ones(78)
normed_probs_first = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs))
data = np.random.multinomial(total_reads, normed_probs_first)
gibbs_data = [[] for i in range(len(X_probs))]

for i in range(10000):
    if (i+1) % 500 == 0:
        print 'Finished {} iterations'.format(i)
    for exon in range(len(X_probs)):
        test = []
        pdf = []
        for value in cnv_support:
            cnv[exon] = value
            # get new normed probabilities given test value
            normed_probs_test = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs)) 
            pdf.append(mpmath.fprod(np.power(normed_probs_test, data)))
            log_likelihood =  np.sum(np.multiply(np.log(normed_probs_test), data))
            test.append(log_likelihood)
        pdf = pdf / mpmath.fsum(pdf)
        new_cnv = np.argmax(test) + 1 if use_log else np.random.choice(cnv_support, 1 , p=pdf) [0]
        # set new copy number for exon and save data
        cnv[exon] = new_cnv
        gibbs_data[exon].append(new_cnv)
        
        # regenerate read data after each new sampling of CNV 
        normed_probs = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs))
        data = np.random.multinomial(total_reads, normed_probs)

[2 1 2 3 1 3 3 3 3 2 3 2 1 1 1 1 3 3 1 3 3 1 2 1 1 2 2 3 3 2 2 1 1 1 1 3 3
 2 2 1 1 2 3 3 1 3 3 3 2 1 2 3 2 1 3 3 2 2 2 3 2 2 3 3 1 2 3 2 3 2 2 2 1 2
 3 1 2 3]


TypeError: unsupported operand type(s) for /: 'list' and 'mpf'

In [5]:
gibbs_data_results = []
# get proportions using burn-in of 1000 iterations 
for exon in gibbs_data:
    hold = [exon[1000:].count(cnv) / float(9000) for cnv in cnv_support]
    gibbs_data_results.append(hold)

NameError: name 'gibbs_data' is not defined

In [123]:
gibbs_df = pd.DataFrame(gibbs_data_results, columns =['copy_1', 'copy_2', 'copy_3']).reset_index()
gibbs_df.rename(columns={'index': 'Exon'}, inplace=True)
gibbs_df.Exon = gibbs_df.Exon + 1

In [124]:
gibbs_df.head(10)

,Exon,copy_1,copy_2,copy_3
0,1,0.000000,0.628889,0.371111
1,2,0.439889,0.280333,0.279778
2,3,0.303444,0.352556,0.344000
3,4,0.244444,0.409889,0.345667
4,5,0.384778,0.328444,0.286778
5,6,0.344222,0.310778,0.345000
6,7,0.410000,0.297667,0.292333
7,8,0.361222,0.322444,0.316333
8,9,0.387889,0.325889,0.286222
9,10,0.247333,0.352556,0.400111


In [3]:
def generate_gibbs_df(cnv_support, X_probs, cnv=None, total_reads=5000, use_log=True, iterations=10000):
    if not cnv:
        cnv = np.random.randint(low=1, high=4, size=78) 
    print cnv 
    normed_probs_first = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs))
    data = np.random.multinomial(total_reads, normed_probs_first)
    gibbs_data = [[] for i in range(len(X_probs))]
    
    for i in range(iterations):
        if (i+1) % 500 == 0:
            print 'Finished {} iterations'.format(i)
        for exon in range(len(X_probs)):
            test = np.zeros(3)
            pdf = []
            for value in cnv_support:
                cnv[exon] = value
                # get new normed probabilities given test value
                normed_probs_test = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs)) 
#                 pdf.append(mpmath.fprod(np.power(normed_probs_test, data)))
                log_likelihood =  np.sum(np.multiply(np.log(normed_probs_test), data))
                test[value - 1] = log_likelihood
            test = test - min(test)
            sample_probs = np.exp(test)
            sample_probs = sample_probs / sum(sample_probs)
#             pdf = mpmath.matrix(pdf) / mpmath.fsum(pdf) if not use_log else []
#             pdf_list = np.array(pdf.tolist(), dtype=float)
#             pdf_list = [x[0] for x in pdf_list]
            new_cnv = np.random.choice(cnv_support, p = sample_probs)
            #new_cnv = np.argmax(test) + 1 #if use_log else np.random.choice(cnv_support, 1 , p=pdf_list)[0]
            # set new copy number for exon and save data
            cnv[exon] = new_cnv
            gibbs_data[exon].append(new_cnv)

            # regenerate read data after each new sampling of CNV 
#             normed_probs = np.multiply(cnv, X_probs) / np.sum(np.multiply(cnv, X_probs))
#             data = np.random.multinomial(total_reads, normed_probs)

    gibbs_data_results = []
    # get proportions using burn-in of 1000 iterations 
    for exon in gibbs_data:
        hold = [exon[1000:].count(cnv) / float(9000) for cnv in cnv_support]
        gibbs_data_results.append(hold)
    
    gibbs_df = pd.DataFrame(gibbs_data_results, columns =['copy_{}'.format(cnv) for cnv in cnv_support]).reset_index()
    gibbs_df.rename(columns={'index': 'Exon'}, inplace=True)
    gibbs_df.Exon = gibbs_df.Exon + 1
    
    return gibbs_data, gibbs_data_results, gibbs_df

In [4]:
gibbs_data2, gibbs_data_results2, gibbs_df2 = generate_gibbs_df(cnv_support, X_probs, use_log=True)

NameError: name 'cnv_support' is not defined

In [179]:
gibbs_df2.head(10)

,Exon,copy_1,copy_2,copy_3
0,1,1.0,0.0,0.0
1,2,1.0,0.0,0.0
2,3,0.0,0.0,1.0
3,4,1.0,0.0,0.0
4,5,0.0,1.0,0.0
5,6,0.0,0.0,1.0
6,7,1.0,0.0,0.0
7,8,0.0,0.0,1.0
8,9,0.0,1.0,0.0
9,10,0.0,0.0,1.0


In [180]:
gibbs_data2[0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
